In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import snowflake.snowpark.functions as F

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
t_org = session.table('T_ORGANIZATIONS').select(F.col("ORGN_ID"),\
                                                F.col("ORGANIZATION_NAME"),\
                                                F.col("DSS_OFFICE_TYPE"))
t_org_org = session.table('T_ORGANIZATION_ORGANIZATION').select(F.col("ORGN_ORGN_ID"),\
                                                F.col("ORGN_ORGN_SUBORGN_ID"))


# Define the schema structure
schema = {
    "res_orgn_id": [],
    "res_unit_hr_orgn_id": [],
    "res_unit_hr_name": [],
    "res_parent_unit_orgn_id": [],
    "res_parent_unit_name": [],
    "res_parent_area_orgn_id": [],
    "res_parent_area_name": [],
    "res_parent_regional_orgn_id": [],
    "res_parent_regional_name": [],
    "res_parent_central_orgn_id": [],
    "res_parent_central_name": []
}

# Create an empty DataFrame with the schema
res_table = pd.DataFrame(schema)


# print(res_table)
# print(len(res_table))

# new_row = {
#             "orgn_id": 1,
#             "unit_hr_orgn_id": 1 * 10,
#             "unit_hr_name": f"Unit 1",
#             "parent_unit_orgn_id": 1 * 100,
#             "parent_unit_name": f"Parent Unit 1",
#             "parent_area_orgn_id": 1 * 1000,
#             "parent_area_name": f"Area 1",
#             "parent_regional_orgn_id": 1 * 10000,
#             "parent_regional_name": f"Regional 1",
#             "parent_central_orgn_id": 1 * 100000,
#             "parent_central_name": f"Central 1"
#         }
# res_table.loc[len(res_table)] = new_row

# print(res_table)

In [ ]:
# sql = """select distinct dss_office_type,orgn_orgn_suborgn_id orgn_id
# from t_organizations o inner join t_organization_organization oo
#     on o.orgn_id = oo.orgn_orgn_suborgn_id
# where dss_office_type in (103228,159938,
#                                 99585,99586,99587)
#         --and orgn_orgn_suborgn_id in (50213)
#         --and orgn_orgn_suborgn_id in (49820,
# --49826,
# --49821,
# --49824,
# --49823,
# --49822)
# --limit 1000
#     """

sql = """
    select distinct dss_office_type, orgn_id
    from t_organizations o 
    where dss_office_type in (103228,159938,
                                99585,99586,99587);
"""

my_df = session.sql(sql)

my_df

In [ ]:
new_rows = []

orgn_name_dict = {row['ORGN_ID']: row['ORGANIZATION_NAME'] for row in t_org.to_local_iterator()}

for row in my_df.to_local_iterator():
    orgn_id = row['ORGN_ID']
    dss_office_type = row['DSS_OFFICE_TYPE']
    unit_hr_orgn_id = 0
    unit_hr_name = ''
    parent_unit_orgn_id = 0
    parent_unit_name = ''
    parent_area_orgn_id = 0
    parent_area_name = ''
    parent_regional_orgn_id = 0
    parent_regional_name = ''
    parent_central_orgn_id = 0
    parent_central_name = ''

    sql1 = f"""
        WITH RECURSIVE org_hierarchy AS (
            SELECT 
                orgn_orgn_id,
                orgn_orgn_suborgn_id
            FROM 
                t_organization_organization
            WHERE 
                orgn_orgn_suborgn_id =  {orgn_id}  
            UNION ALL
            SELECT 
                t.orgn_orgn_id,
                t.orgn_orgn_suborgn_id
            FROM 
                t_organization_organization t
            JOIN 
                org_hierarchy oh ON t.orgn_orgn_suborgn_id = oh.orgn_orgn_id
                inner join t_organizations too on  t.orgn_orgn_id = too.orgn_id
        )
         
        SELECT oh.*,dss_office_type FROM org_hierarchy oh
        inner join t_organizations too on too.orgn_id = oh.orgn_orgn_id
        union 
        select {orgn_id} ,null,{dss_office_type}
    """
    res_df = session.sql(sql1)
    res_df


    # res_df = res_df.join(t_org ,res_df["ORGN_ORGN_ID"] == t_org["ORGN_ID"],'left')
    
    unit_df = res_df[res_df["DSS_OFFICE_TYPE"] == '103228']
    area_df = res_df[res_df["DSS_OFFICE_TYPE"] == '99587']
    regional_df = res_df[res_df["DSS_OFFICE_TYPE"] == '99586']
    central_df = res_df[res_df["DSS_OFFICE_TYPE"] == '99585']
    
    unit_df

    # area_df

    # regional_df

    # central_df

    parent_unit_orgn_id = unit_df.first()["ORGN_ORGN_ID"] if unit_df.first() is not None else None
    parent_unit_name = None if parent_unit_orgn_id is None else orgn_name_dict[parent_unit_orgn_id]

    # print(parent_unit_orgn_id)
    parent_area_orgn_id = area_df.first()["ORGN_ORGN_ID"] if area_df.first() is not None else None
    parent_area_name = None if parent_area_orgn_id is None else orgn_name_dict[parent_area_orgn_id]
    # print(parent_area_orgn_id)

    parent_regional_orgn_id = regional_df.first()["ORGN_ORGN_ID"] if regional_df.first() is not None else None
    parent_regional_name = None if parent_regional_orgn_id is None else orgn_name_dict[parent_regional_orgn_id]
    # print(parent_regional_orgn_id)

    parent_central_orgn_id = central_df.first()["ORGN_ORGN_ID"] if central_df.first() is not None else None
    parent_central_name = None if parent_central_orgn_id is None else orgn_name_dict[parent_central_orgn_id]


    new_row = {
        'res_orgn_id': orgn_id,
        'res_unit_hr_orgn_id': unit_hr_orgn_id,
        'res_unit_hr_name': unit_hr_name,
        'res_parent_unit_orgn_id': parent_unit_orgn_id,
        'res_parent_unit_name': parent_unit_name,
        'res_parent_area_orgn_id': parent_area_orgn_id,
        'res_parent_area_name': parent_area_name,
        'res_parent_regional_orgn_id': parent_regional_orgn_id,
        'res_parent_regional_name': parent_regional_name,
        'res_parent_central_orgn_id': parent_central_orgn_id,
        'res_parent_central_name': parent_central_name
    }

    new_rows.append(new_row)

# Create the DataFrame from the list of rows
res_table = pd.DataFrame(new_rows)
res_table

In [ ]:
org_structure = session.table("ORGANIZATION_STRUCTURE1")
print(org_structure.schema)
print(new_rows)
res_table1 = session.create_dataframe(res_table)
print(res_table1.schema)
res_table1['"res_orgn_id"']

org_structure.merge(res_table1 ,(org_structure["ORGN_ID"] == res_table1['"res_orgn_id"']),\
                   [F.when_not_matched().insert({"ORGN_ID":res_table1['"res_orgn_id"'], "UNIT_HR_ORGN_ID":res_table1['"res_unit_hr_orgn_id"'], "UNIT_HR_NAME":res_table1['"res_unit_hr_name"'],\
                                                 "PARENT_UNIT_ORGN_ID":res_table1['"res_parent_unit_orgn_id"'], "PARENT_UNIT_NAME":res_table1['"res_parent_unit_name"'], \
                                                 "PARENT_AREA_ORGN_ID":res_table1['"res_parent_area_orgn_id"'], "PARENT_AREA_NAME":res_table1['"res_parent_area_name"'],\
                                                 "PARENT_REGIONAL_ORGN_ID":res_table1['"res_parent_regional_orgn_id"'], "PARENT_REGIONAL_NAME":res_table1['"res_parent_regional_name"'],\
                                                 "PARENT_CENTRAL_ORGN_ID":res_table1['"res_parent_central_orgn_id"'], "PARENT_CENTRAL_NAME":res_table1['"res_parent_central_name"']})])